### Schema Alignment Example

In [2]:
import keras
import urllib2

import pandas as pd

from hashlib import md5
from pprint import pprint
from bs4 import BeautifulSoup
from sklearn.cluster import DBSCAN

import sys
sys.path.append('/Users/BenJohnson/projects/what-is-this/wit/')
from wit import *

Using Theano backend.


#### Task

We attempt to approach the problem of "schema alignment": given two datasets that contain roughly the same kinds of data, merge them into a single dataset such that columns are appropriately aligned.  This is a generic problem that comes up when we're trying to merge data, but in this example we explore it's application to data attained via web scraping.
 
Specifically, we'll try to do this with data scraped from message board forums, which contains data types such as

    username
    post body
    signature
    date
    ...

So the task is to align the username field in the first dataset with the username field in the second dataset, etc.  The usual approach is to write/configure a parser for each schema, which includes a manual mapping of data fields from the HTML page to 

#### Data

We'll use data scraped by Hyperion Gray from 

    remingtonsociety.com
    marauderairrifle.com

A post in the first set looks like:
```html
<tr class="row1">
    <td align="center" class="row" valign="top">
        <a name="p6755"></a>
        <div class="postauthor">Noel</div>
        <div class="postdetails">
                <br/> <b>Joined:</b> Tue Jun 26, 2007 8:48 pm 
                <br/> <b>Posts: </b> 112
        </div>
        <img alt="" height="1" src="spacer.gif" width="120"/>
    </td>
    <td class="row" height="25" valign="top" width="100%">
        <div style="float: right;"></div>
        <div class="postsubject">
            <a href="./viewtopic.php?p=6755#p6755">
                <img alt="Post" height="9" src="icon_topic_latest.gif" title="Post" width="13"/>
            </a> 
            Remington 12A wanted
        </div>
        <div class="postbody">
            Looking for a 12A with condition. Bore must be very good plus, external at 80% or better please.
        </div>
        <g:plusone size="small"></g:plusone>
        <br clear="all"/><br/>
    </td>
</tr>
```

A post in the second set looks like:
```html
<div class="post bg2" id="p38336">
    <div class="inner">
        <span class="corners-top">
            <span></span>
        </span>
        <div class="postbody">
            <h3 class="first">
                <a href="#p38336">Re: Is this what we need for 22 M-rod accuracy?</a>
            </h3>
            <p class="author">
                <a href="./viewtopic.php?p=38336#p38336">
                    <img alt="Post" height="9" src="icon_post_target.gif" title="Post" width="11"/>
                </a>
                by 
                <strong>
                    <a href="./memberlist.php?mode=viewprofile&amp;u=1195">
                        RayK
                    </a>
                </strong> 
                » Tue May 28, 2013 9:18 pm 
            </p>
            <div class="content">
                Thanks for the new photos!  Those are great and clearly show the idea.<br/><br/>Mine currently look just like that... <br/><br/>Ray
            </div>
            <div class="signature" id="sig38336">
                25 W-rod | 8 shots @ 70 FPE -2% | 3100 fill<br/>25 M-rod
            </div>
        </div>
        <dl class="postprofile" id="profile38336">
            <dt>
                <a href="./memberlist.php?mode=viewprofile&amp;u=1195">RayK</a>
            </dt>
            <dd></dd>
            <dd><strong>Posts:</strong> 5311</dd>
            <dd><strong>Joined:</strong> Sat Jun 05, 2010 3:19 pm</dd>
        </dl>
        <div class="back2top">
            <a class="top" href="#wrap" title="Top">
                Top
            </a>
        </div>
        <span class="corners-bottom"><span></span></span>
    </div>
</div>
```

They have somewhat similar structure, and actually look like maybe they were generated by a different configuration of the same forum building software.  However, they're sufficiently different that they cannot be parsed with the same CSS selectors or regular expressions.

#### Approach

There are a few stages to this problem:

    a) Finding 'atomic data elements' in the raw HTML page.  
        - i.e. separating the posts on a page that has multiple posts
    
    b) Finding the 'data fields' within an atomic data element
        - i.e. finding the locations in the DOM that contain data points we're interested in capturing.
    
    c) Merging 'data fields' across schema

a) and b) are difficult in their own right, and we implement simple, non-optimal solutions.  For a), we totally punt and just do it manually.  For b):

    - find all nodes that contain text in the corpus of posts
    - find CSS paths of the format
    
        tag1[class1, ...] > tag2[class2, ...] > ... > tag_n[class_m, ...]
    
      to each of these nodes.  From here on, when we say "CSS path", we mean CSS path w/ classes, w/o indices.
    - assume data in nodes with the same CSS path are "equivalent".  

The last step is an assumption that could be wrong in both directions: data elements with the same CSS path could be different latent variables, and data elements with different CSS paths could be the same latent variables.  These issues could be mitigated by applying more advanced wrapper induction techniques, but they do not turn out to be too damaging to the current approach.  In fact, the process of merging two schemas also merges classes within the same schema (for better or for worse).

Set parameters

In [3]:
num_features = 75  # in characters
max_len      = 350 # in characters
formatter    = KerasFormatter(num_features, max_len)

Load data from two forums with different schemas.

```
origin : domain that record comes from
hash   : md5 hash of the path to the elements in the DOM (including classes)
id     : record number within origin + md5 hash of origin field
obj    : text content of DOM node
src    : corresponding raw HTML markup
```

In [4]:
df = pd.read_csv('data/simple-forum-dataset.csv')
df.tail()

,hash,id,obj,src
23229,e5316-48774,2662-48774,"by jer295 Sat Mar 09, 2013 9:34 pm","<p class=""author""><a href=""./viewtopic.php?p=3..."
23230,3122c-48774,2662-48774,Standard oring are to fat to work the one that...,"<div class=""content"">Standard oring are to fat..."
23231,df7d2-48774,2662-48774,jer295,"<a href=""./memberlist.php?mode=viewprofile&amp..."
23232,5fd24-48774,2662-48774,Posts: 8,<dd><strong>Posts:</strong> 8</dd>
23233,5fd24-48774,2662-48774,"Joined: Thu Mar 07, 2013 7:33 am","<dd><strong>Joined:</strong> Thu Mar 07, 2013 ..."


Make training set, which will consist of triplets of points

    (anchor, positive, negative)
        
where 

    - anchor and positive are of the same class
    - anchor and negative are of different classes
    


We'll minimize the loss function

    max(0, distance(anchor, positive) - distance(anchor, negative) + margin)

In this case, we use

    distance(x, y) = 1 - cosine_similarity(x, y)

but 

    distance(x, y) = euclidean_distance(x, y)
    
also seems to work in general.

In [7]:
train     = make_triplet_train(df, N = 600)
trn, levs = formatter.format(train, ['obj'], 'hash')



  + 459fb-1706c
  + 520fa-1706c
  + 9ead9-1706c
  + 34883-1706c
  + 2e98d-1706c
  + 2bb6e-1706c
  + bf07b-1706c
  + 8fbdb-1706c
  + a1c8c-48774
  + e5316-48774
  + 3122c-48774
  + df7d2-48774
  + 5fd24-48774
  + 58faa-48774
  + 0dba3-48774
  + 9acc3-48774
  + 6d664-48774
  + 20a50-48774
  + b9976-48774




Define and train the model.  This should take several minutes on the current dataset.

In [8]:
classifier = TripletClassifier(trn, levs)
classifier.fit(batch_size = 250, nb_epoch = 3)

Create set of all unique records (without duplicates), then project them using the model.  The projection should take tens of seconds.

In [11]:
unq = df.copy()
del unq['id']
unq = unq.drop_duplicates()

awl, _ = formatter.format(unq, ['obj'], 'hash')
preds  = classifer.predict(awl['x'][0], verbose = True)

14314/14314 [==============================] - 13s    


Cluster the projections using DBSCAN, then interpret the cluster to be the aligned datatypes.  

Though the string similarity was performed on just the HTML text content, we show the classes with HTML markup so we can easily verify the cluster assignments.  Notice that some of the clusters include hashes from both schemas, while others only include hashes from a single schema -- this is desired behavior, as there is not a 1-to-1 mapping between the two schemas.

In [31]:
db = DBSCAN(eps = .1, min_samples = 50).fit(preds)

# Observation cluster assignments by hash
res         = unq.hash.groupby(db.labels_).apply(lambda x: x.value_counts()).reset_index()
res.columns = ('cluster', 'hash', 'cnt')

# Discard small clusters, or points that were assigned to None cluster
good_res = res[(res.cnt > 100) & (res.cluster > -1)]
eqv = list(good_res.groupby('cluster').hash.apply(lambda x: list(x)))
eqv = map(eval, np.unique(map(str, eqv)))

print '\n-----------------------'
print 'clusters, without markup'
print_eqv(eqv, df, path = 'obj')

print '\n--------------------------------------'
print 'clusters, with markup for verification'
print_eqv(eqv, df, path = 'src')




height has been deprecated.


-----------------------
clusters, without markup

 --- 

['3122c-48774', '9acc3-48774', '34883-1706c', '8fbdb-1706c']


3122c-48774	(1428 rows)
10253    ShawnHu wrote:How about a hammer tapping servi...
14293    I am totally against this, and I think you are...
21204    My bolt broken last night. I wanted to know if...
17490    This line of scopes have alot of potential and...
18197    HI guys, this morning I decided to try and mak...
Name: obj, dtype: object
9acc3-48774	(1209 rows)
13566    The trigger screws do not affect the hammer tr...
17055    thanks for the response man I have been using ...
7474     All my misses are @ close range. So I guess I ...
6539     bstaley wrote:Got a chance to try the heat shr...
15720            Shawnhu,PM just sent with my contact info
Name: obj, dtype: object
34883-1706c	(975 rows)
838     i have one too without markings on barrel . th...
5784    Hello, looks like a nice board to which I hope...
866     I would like to